In [ ]:
%run linearelliptic_block_swipdg.ipynb

In [ ]:
import numpy as np

In [ ]:
print('estimating error ...')

over_integration = 2

local_sizes = [block_space.local_space(ii).size() for ii in range(block_space.num_blocks)]
local_starts = [int(np.sum(local_sizes[:ii])) for ii in range(block_space.num_blocks)]
local_starts.append(block_space.mapper.size)
localized_u_h_np = np.array(u_h_vector, copy=False)
localized_u_h_np = [localized_u_h_np[local_starts[ii]:local_starts[ii+1]] for ii in range(block_space.num_blocks)]
localized_u_h = [Vector(block_space.local_space(ii).size(), 0.) for ii in range(block_space.num_blocks)]
for ii in range(block_space.num_blocks):
    tmp = np.array(localized_u_h[ii], copy=False)
    tmp[:] = localized_u_h_np[ii][:]
local_boundary_info = make_subdomain_boundary_info(grid, {'type': 'xt.grid.boundaryinfo.boundarysegmentindexbased',
                                                          'default': 'dirichlet',
                                                          'neumann': '[{} {}]'.format(inner_boundary_id, inner_boundary_id+1)})

# the nonconformity indicator
from dune.gdt import (
    make_ESV2007_nonconformity_product_dd_subdomain_part_dd_subdomain_oversampled_part
        as make_local_nonconformity_product
)

eta_nc_squared = 0.
for ii in range(grid.num_subdomains):
    local_u_h_ii = make_discrete_function(block_space.local_space(ii), localized_u_h[ii], 'u_h'.format(ii))

    neighborhood = grid.neighborhood_of(ii)
    neighborhood_space = block_space.restricted_to_neighborhood(neighborhood)
    restricted_u_h_vector = neighborhood_space.project_onto_neighborhood([localized_u_h[jj] for jj in neighborhood],
                                                                         neighborhood)
    restricted_u_h = make_discrete_function(neighborhood_space, restricted_u_h_vector, 'u_h')
    local_nonconformity_product = make_local_nonconformity_product(grid, local_boundary_info,
                                                                   lambda_, kappa,
                                                                   restricted_u_h, restricted_u_h,
                                                                   over_integration,
                                                                   ii, ii)
    eta_nc_squared += local_nonconformity_product.apply2()
print('  nonconformity indicator: is {} (should be close to 1.66e-01)'.format(np.sqrt(eta_nc_squared)))